In [1]:
%reload_ext autoreload
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
import os
import sqlalchemy as sa

import database as db
from images import copy_images, get_list_of_images_invalid, save_metadata, select_images
from models import County, DataTrustedIdentifier

In [3]:
engine, session=db.connect()
engine.echo=False

2023-04-02 16:40:02,840 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2023-04-02 16:40:02,840 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-04-02 16:40:02,841 INFO sqlalchemy.engine.Engine select current_schema()
2023-04-02 16:40:02,842 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-04-02 16:40:02,843 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2023-04-02 16:40:02,843 INFO sqlalchemy.engine.Engine [raw sql] {}


In [4]:
list_minimum_image=[5, 10, 20]
list_color=['rgb', 'grayscale']
list_image_size=['256', '400', '512']
list_regions=['Norte', 'Nordeste', 'Centro-Oeste', 'Sul', 'Sudeste']
path_out = '/home/xandao/Imagens/region_dataset'
list_images_invalid=get_list_of_images_invalid()
list_images_invalid=list_images_invalid['barcode']

In [7]:
for color in list_color:
    for region in list_regions:
        for image_size in list_image_size:
            for minimum_image in list_minimum_image:
                query = session.query(County).filter(County.regiao.__eq__(region)).distinct().all()

                state_unaccented_lower, uf_unaccented_lower, county_unaccented_lower = db.get_state_uf_county(query)

                list_level = []
                list_path_images_final = []
                list_seq_final = []
                list_count_samples = []

                level=DataTrustedIdentifier.specific_epithet_trusted
                out=os.path.join(path_out, color.upper(), level.name, region, image_size, str(minimum_image))

                condition = sa.and_(DataTrustedIdentifier.country_trusted.__eq__('Brasil'),
                                    level.is_not(None),
                                    sa.or_(sa.and_(uf_unaccented_lower,
                                           state_unaccented_lower,
                                           county_unaccented_lower),
                                           sa.and_(uf_unaccented_lower,
                                           county_unaccented_lower),
                                           sa.and_(state_unaccented_lower,
                                           county_unaccented_lower),
                                           uf_unaccented_lower,
                                           state_unaccented_lower)
                                    )

                query = db.get_records_group_by_level(condition, level, minimum_image, session)
                print('region: %s color: %s image_size: %s minimum_image: %d' % (region, color.upper(), image_size, minimum_image))
                select_images(color, image_size, list_count_samples, list_level, list_images_invalid, list_path_images_final, list_seq_final, minimum_image, session, query)

                list_f = copy_images(list_level, list_path_images_final, out)
                save_metadata(color, image_size, level, list_count_samples, list_f, list_level, list_path_images_final, list_seq_final, minimum_image, out, session, region=region)

region: Norte color: RGB image_size: 256 minimum_image: 5
total of levels: 107 total of images: 2637


,levels,count,dst,paths,seq
0,aduncum,48,/home/xandao/Imagens/region_dataset/RGB/specif...,[/media/xandao/c2f58d30-ff2c-47f7-95af-91ad6fd...,"[11893, 12586, 14141, 30236, 30237, 30247, 302..."
1,aequale,9,/home/xandao/Imagens/region_dataset/RGB/specif...,[/media/xandao/c2f58d30-ff2c-47f7-95af-91ad6fd...,"[12577, 30534, 30546, 36793, 43763, 43769, 457..."
2,alata,8,/home/xandao/Imagens/region_dataset/RGB/specif...,[/media/xandao/c2f58d30-ff2c-47f7-95af-91ad6fd...,"[26378, 26385, 26457, 37031, 43122, 43125, 431..."


,seq,genus,specific_epithet,genus_trusted,specific_epithet_trusted,country_trusted,country,county,state_province,urls
0,1363,Peperomia,glabella,Peperomia,glabella,Brasil,Brazil,None,Acre,[https://storage.googleapis.com/cria-zoomify/h...
1,44254,Piper,coruscans,Piper,coruscans,Brasil,Brasil,Porto Velho,Rondônia,[https://storage.googleapis.com/cria-zoomify/h...
2,13605,Piper,arboreum,Piper,arboreum,Brasil,Brasil,None,Pará,[https://storage.googleapis.com/cria-zoomify/h...


,0
dst,/home/xandao/Imagens/region_dataset/RGB/specif...
color,rgb
image_size,256
minimum_image,5
level_name,specific_epithet_trusted
region,Norte
levels,107
total,2637
average,24.64


region: Norte color: RGB image_size: 256 minimum_image: 10
total of levels: 68 total of images: 2353


...

,0
dst,/home/xandao/Imagens/region_dataset/GRAYSCALE/...
color,grayscale
image_size,512
minimum_image,10
level_name,specific_epithet_trusted
region,Sudeste
levels,68
total,2536
average,37.29


region: Sudeste color: GRAYSCALE image_size: 512 minimum_image: 20
total of levels: 42 total of images: 2211


,levels,count,dst,paths,seq
0,aduncum,91,/home/xandao/Imagens/region_dataset/GRAYSCALE/...,[/media/xandao/c2f58d30-ff2c-47f7-95af-91ad6fd...,"[7742, 7757, 8700, 8705, 8711, 9435, 9439, 957..."
1,alata,28,/home/xandao/Imagens/region_dataset/GRAYSCALE/...,[/media/xandao/c2f58d30-ff2c-47f7-95af-91ad6fd...,"[6857, 6861, 6862, 6863, 6864, 7565, 7859, 110..."
2,amalago,111,/home/xandao/Imagens/region_dataset/GRAYSCALE/...,[/media/xandao/c2f58d30-ff2c-47f7-95af-91ad6fd...,"[2798, 7741, 7908, 7928, 8751, 8757, 8758, 876..."


,seq,genus,specific_epithet,genus_trusted,specific_epithet_trusted,country_trusted,country,county,state_province,urls
0,19770,Ottonia,anisum,Piper,anisum,Brasil,Brasil,Guapimirim,Rio de Janeiro,[https://storage.googleapis.com/cria-zoomify/h...
1,7333,Piper,corcovadense,Piper,grazielae,Brasil,Brazil,Tapira,São Paulo,[https://storage.googleapis.com/cria-zoomify/h...
2,21142,Ottonia,propinqua,Piper,grazielae,Brasil,Brasil,Limeira,São Paulo,[https://storage.googleapis.com/cria-zoomify/h...


,0
dst,/home/xandao/Imagens/region_dataset/GRAYSCALE/...
color,grayscale
image_size,512
minimum_image,20
level_name,specific_epithet_trusted
region,Sudeste
levels,42
total,2211
average,52.64


In [6]:
session.close()
engine.dispose()